In [102]:
import pickle
# from functools import partial
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['figure.figsize'] = (9, 9)
# import seaborn as sns
# import plotly.express as px

from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.dummy import DummyClassifier
# feature selection
from sklearn.feature_selection import chi2, SelectPercentile, SelectKBest
# model selection
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.metrics import (r2_score, mean_squared_error, accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve, precision_recall_curve, make_scorer,
                             confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay)

# models
from statsmodels.tsa.arima.model import ARIMA
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor,ExtraTreesRegressor,ExtraTreesClassifier
from xgboost import XGBClassifier, XGBRegressor
import xgboost as xgb
# model explainability
import shap

# enable reload of changed files
%reload_ext autoreload
%autoreload 2

# plot inline
#%matplotlib inline

# make it pretty
#from rich.console import Console
#from rich import inspect
#con = Console()
#ins = partial(inspect, methods=True)
#%reload_ext rich
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, OrdinalEncoder, FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import (precision_recall_curve,PrecisionRecallDisplay)

def mod_eval(xtrain,xtest,ytrain,ytest,xfc,regressor,probs=0):
    kpi=pd.DataFrame(index=['mae','rmse','mape'])
    fc=pd.DataFrame(index=list(range(1,25)))
    
    regressor[1].fit(xtrain,ytrain)
    fctest=regressor[1].predict(xtest)
    mae = np.mean(abs(fctest - ytest.values))
    mape=(fctest - ytest.values)/ytest.values;
    mape=100*np.mean(abs(mape[np.isfinite(mape)]))
    rmse = np.sqrt(np.mean((fctest - ytest.values) ** 2))
    
    kpi.loc[:,regressor[0]]=[mae,rmse,mape]
    fc.loc[:,regressor[0]]=regressor[1].predict(xfc)
    
    if probs==1:
        p=prob_eval(regressor,xfc)
        fc=pd.concat([fc,p],axis=1)
    return kpi, fc

def prob_eval(regressor,xfc):
    fc=pd.DataFrame(index=list(range(1,25)))

In [2]:
# lectura de variables para pronostico de PML y OVEMEM
rawdata = pd.read_excel('data\input_vars.xlsx')
print(rawdata.dtypes,rawdata.isnull().sum())

Fecha      datetime64[ns]
Mes                 int64
Dia                 int64
Diasem              int64
Hora                int64
Fest                int64
MDA               float64
MTR               float64
Spread            float64
Bin                 int64
MW                float64
SIN               float64
FC                  int64
MRO               float64
TEM                 int64
EO                float64
FV                float64
RES               float64
MDA2              float64
MDA3              float64
MDA4              float64
MW2               float64
MW3               float64
MW4               float64
EO1               float64
FV1               float64
HSC1              float64
SIN2              float64
Spread4           float64
FC1                 int64
Bin3                int64
Binh3               int64
Bin7                int64
Bin14               int64
dtype: object Fecha      0
Mes        0
Dia        0
Diasem     0
Hora       0
Fest       0
MDA        0
MTR        

In [44]:
df_datestr=rawdata[['Fecha','Hora']]
df_datetime=pd.DataFrame({'Fecha': rawdata.Fecha
                         +pd.to_timedelta(rawdata.Hora,unit='h')})

df_pml=rawdata.copy()
df_pml['Fecha']=df_datetime['Fecha'].dt.strftime('%d/%m/%Y %H')
df_pml.set_index('Fecha',inplace=True)


In [47]:
date_rng=['01/07/2022 01','01/01/2023 00']

In [61]:
#Selección de datos que se utilizan en entrenamiento y prueba de algoritmos
hist_data=df_pml['01/07/2022 01':'01/01/2023 00']

# Seleccion de datos que se utilizan en validación
fc_data=df_pml['01/01/2023 01':]
fc_data

,Mes,Dia,Diasem,Hora,Fest,MDA,MTR,Spread,Bin,MW,...,EO1,FV1,HSC1,SIN2,Spread4,FC1,Bin3,Binh3,Bin7,Bin14
Fecha,,,,,,,,,,,,,,,,,,,,,
01/07/2022 01,7,1,6,1,0,1208.15,1228.09,19.94,1,837.012,...,2119.700,0.000,6.545,41518.321086,-40.66,1,0,1,0,0
01/07/2022 02,7,1,6,2,0,1188.07,1185.21,-2.86,0,821.039,...,2137.464,0.000,6.545,41518.321086,12.16,1,0,0,0,0
01/07/2022 03,7,1,6,3,0,1174.54,1159.35,-15.19,0,796.472,...,2137.813,0.000,6.545,41518.321086,-37.54,1,0,0,0,0
01/07/2022 04,7,1,6,4,0,1154.17,1138.93,-15.24,0,790.709,...,2060.523,0.000,6.545,41518.321086,-28.50,1,0,0,0,0
01/07/2022 05,7,1,6,5,0,1149.83,1127.51,-22.32,0,793.122,...,1988.454,0.000,6.545,41518.321086,-9.43,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31/12/2022 20,12,31,7,20,1,413.40,452.24,38.84,1,553.496,...,1236.534,0.023,2.200,30159.560185,-38.22,0,1,0,1,0
31/12/2022 21,12,31,7,21,1,343.85,423.35,79.50,1,533.799,...,1204.235,0.000,2.200,30159.560185,-42.09,0,1,0,1,0
31/12/2022 22,12,31,7,22,1,330.45,412.99,82.54,1,512.987,...,1138.072,0.000,2.200,30159.560185,-47.74,0,1,1,1,0


In [72]:
X,y=hist_data[['Hora','Dia','MDA2','MDA3','MDA4','MW2','MW3','MW4','HSC1','TEM','EO1','FV1']],hist_data[['MDA']]

In [73]:
y

,MDA
Fecha,
01/07/2022 01,1208.15
01/07/2022 02,1188.07
01/07/2022 03,1174.54
01/07/2022 04,1154.17
01/07/2022 05,1149.83
...,...
31/12/2022 20,413.40
31/12/2022 21,343.85
31/12/2022 22,330.45


In [74]:
# Input vars for LMP
# Selección de variable a pronosticar y datos de entrada a modelos
X_train,X_test,y_train,y_test=train_test_split(X,y,shuffle=False)
X_train

,Hora,Dia,MDA2,MDA3,MDA4,MW2,MW3,MW4,HSC1,TEM,EO1,FV1
Fecha,,,,,,,,,,,,
01/07/2022 01,1,1,1128.67,1122.843333,1179.13,914.519,919.371542,881.700,6.545,17,2119.700,0.000
01/07/2022 02,2,1,1128.05,1122.843333,1155.25,892.112,919.371542,859.318,6.545,20,2137.464,0.000
01/07/2022 03,3,1,1081.02,1122.843333,1128.81,866.957,919.371542,835.702,6.545,21,2137.813,0.000
01/07/2022 04,4,1,1075.36,1122.843333,1134.40,854.254,919.371542,826.976,6.545,21,2060.523,0.000
01/07/2022 05,5,1,1067.40,1122.843333,1115.40,851.086,919.371542,819.218,6.545,22,1988.454,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
15/11/2022 20,20,15,791.33,659.729583,585.38,824.931,771.822042,908.330,3.595,8,1566.091,0.024
15/11/2022 21,21,15,776.50,659.729583,495.27,823.055,771.822042,894.700,3.595,7,1625.693,0.000
15/11/2022 22,22,15,699.24,659.729583,487.37,807.959,771.822042,880.610,3.595,7,1736.962,0.000


In [78]:
# Create scaler object
ss = StandardScaler()

# Fit scaler to training data
ss.fit(X_train)

# Transform train and test X data and save as new variables
X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)

# Instantiate our logistic regression object
reg = LinearRegresion()

# Fit logistic regression to scaled training data
reg.fit(X_train_scaled, y_train)

# Score logistic regression on scaled testing data
reg.score(X_test_scaled, y_test)

0.5128643490226219

In [79]:
reg = Ridge(alpha=0.7)
# Fit logistic regression to scaled training data
reg.fit(X_train_scaled, y_train)
# Score logistic regression on scaled testing data
reg.score(X_test_scaled, y_test)

0.5128816930737707

In [83]:
reg = Lasso(alpha=0.96)
# Fit logistic regression to scaled training data
reg.fit(X_train_scaled, y_train)
# Score logistic regression on scaled testing data
reg.score(X_test_scaled, y_test)

0.5123994680477679

In [97]:
reg = ElasticNet(alpha=0.425,l1_ratio=0.99)
# Fit logistic regression to scaled training data
reg.fit(X_train_scaled, y_train)
# Score logistic regression on scaled testing data
reg.score(X_test_scaled, y_test)

0.5120062934754929

In [99]:
reg = RandomForestRegressor(n_estimators=100)
# Fit logistic regression to scaled training data
reg.fit(X_train_scaled, y_train.values.ravel())
# Score logistic regression on scaled testing data
reg.score(X_test_scaled, y_test)

0.4881375914747761

In [103]:
reg = ExtraTreesRegressor(n_estimators=100)
# Fit logistic regression to scaled training data
reg.fit(X_train_scaled, y_train.values.ravel())
# Score logistic regression on scaled testing data
reg.score(X_test_scaled, y_test)

0.5309227585716361

In [ ]:
#label=df_pml['MDA']
#mod_vars=[['Hora','Dia','MDA2','MDA3','MDA4','MW2','MW3','MW4'],
#          ['Hora','Dia','MDA2','MDA3','MDA4','MW2','MW3','MW4','HSC1','TEM'],
#          ['Hora','Dia','MDA2','MDA3','MDA4','MW2','MW3','MW4','EO1','FV1']]

# Modelos considerados y variables de salida:
# pml_fc -> Pronostico de PML
# pml_ind -> Indicadores del modelo durante prueba
model_name_list = ['LR','RF', 'ET','RF+GT','RF+EF']
pml_fc=pd.DataFrame(index=list(range(1,25)))
pml_ind=pd.DataFrame(index=['mae','rmse','mape'])

# Regresores utilizados benchmark
lr=['LR',LinearRegression()]
et=['ET',ExtraTreesRegressor(n_estimators=50)]
rf=['RFx',RandomForestRegressor(n_estimators=50)]
log=['LOG',LogisticRegression(solver='liblinear')]
etx=['ET',ExtraTreesClassifier(n_estimators=1000,random_state=42)]
rfx=['RF',RandomForestClassifier(n_estimators=1000,random_state=42)]
log_pml=['LOG+',LogisticRegression(solver='liblinear')]

# Regresores utilizados Grid Search/Cross Validation
lr-gs=['LR',LinearRegression()]
et-gs=['ET',ExtraTreesRegressor()]
rf-gs=['RFx',RandomForestRegressor()]
log-gs=['LOG',LogisticRegression()]
etx-gs=['ET',ExtraTreesClassifier()]
rfx-gs=['RF',RandomForestClassifier()]
log_pml-gs=['LOG+',LogisticRegression()]

In [ ]:
# En todos los modelos se segmentan las series de tiempo para entrenamiento y prueba. 
# La funcion mod_eval realiza el entrenamiento, prueba y pronóstico de MDA 
# Los resultados se almacenan en pml_fc y pml_ind 

# LR
X=df_pml[mod_vars[0]]
X_fc=df_fc[mod_vars[0]]
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size = 0.25,random_state=42)
metrics,forecast=mod_eval(X_train,X_test,y_train,y_test,X_fc,lr)
pml_fc=pd.concat([pml_fc,forecast],axis=1)
pml_ind=pd.concat([pml_ind,metrics],axis=1)

In [ ]:
data1=data.drop(columns=['instant','dteday','yr'],axis=1)
data1

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X=data1.drop('rentals',axis=1)
y=data1['rentals']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=123)
X_test

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,MinMaxScaler
from sklearn.compose import ColumnTransformer

In [ ]:
numeric_transformer=Pipeline(steps=[
    ('scalar',StandardScaler())
])

#categorical_transformer=Pipeline(steps=[
#    ('imputer',SimpleImputer(strategy='constant')),
#    ('encoder',OrdinalEncoder)
#])

In [ ]:
numeric_f=['temp','atemp','hum','windspeed']
categorical_f=['season','mnth','holiday','weekday','workingday','weathersit']

In [ ]:
preprocessor=ColumnTransformer(
    transformers=[
        ('numeric',numeric_transformer,numeric_f),
        ('categorical',categorical_transformer,categorical_f)
    ]
)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
pipeline_m=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('regressor',RandomForestRegressor())
])
pipeline_m

In [ ]:
Model=pipeline_m.fit(X_train,y_train)
print(Model)